In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [4]:
df = pd.read_csv("Movies.csv", encoding='ISO-8859-1')


In [39]:
df.info()
df.describe()
df.isnull().sum()  


<class 'pandas.core.frame.DataFrame'>
Index: 5659 entries, 1 to 15508
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      5659 non-null   object 
 1   Year      5659 non-null   float64
 2   Duration  5659 non-null   int64  
 3   Genre     5659 non-null   object 
 4   Rating    5659 non-null   float64
 5   Votes     5659 non-null   int64  
 6   Director  5659 non-null   object 
 7   Actor 1   5659 non-null   object 
 8   Actor 2   5659 non-null   object 
 9   Actor 3   5659 non-null   object 
dtypes: float64(2), int64(2), object(6)
memory usage: 486.3+ KB


Name        0
Year        0
Duration    0
Genre       0
Rating      0
Votes       0
Director    0
Actor 1     0
Actor 2     0
Actor 3     0
dtype: int64

In [8]:
df.dropna(inplace=True)
df = df[df['Rating'].notnull()]



In [13]:
selected_features = df[['Genre', 'Director', 'Actor 1', 'Actor 2', 'Actor 3', 'Year', 'Duration']]
target = df['Rating']


In [16]:
from sklearn.preprocessing import LabelEncoder

encoders = {}

for col in ['Genre', 'Director', 'Actor 1', 'Actor 2', 'Actor 3']:
    le = LabelEncoder()
    selected_features[col] = le.fit_transform(selected_features[col])
    encoders[col] = le


C:\Users\HP\AppData\Local\Temp\ipykernel_9716\2582407885.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_features[col] = le.fit_transform(selected_features[col])
C:\Users\HP\AppData\Local\Temp\ipykernel_9716\2582407885.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_features[col] = le.fit_transform(selected_features[col])
C:\Users\HP\AppData\Local\Temp\ipykernel_9716\2582407885.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [17]:
X_train, X_test, y_train, y_test = train_test_split(selected_features, target, test_size=0.2, random_state=42)


In [20]:
import re
import numpy as np

def convert_duration(duration):
    try:
        if isinstance(duration, str):
            h = re.search(r'(\d+)h', duration)
            m = re.search(r'(\d+)min', duration)
            hours = int(h.group(1)) if h else 0
            minutes = int(m.group(1)) if m else 0
            return hours * 60 + minutes
        elif isinstance(duration, (int, float)):
            return duration
        else:
            return np.nan
    except:
        return np.nan


In [21]:
df['Duration'] = df['Duration'].apply(convert_duration)


In [22]:
df.dropna(subset=['Duration'], inplace=True)


In [24]:
print(df['Duration'].dtype) 


int64


In [25]:
selected_features = df[['Genre', 'Director', 'Actor 1', 'Actor 2', 'Actor 3', 'Year', 'Duration', 'Votes']].copy()
target = df['Rating']


In [26]:
from sklearn.preprocessing import LabelEncoder

encoders = {}
for col in ['Genre', 'Director', 'Actor 1', 'Actor 2', 'Actor 3']:
    le = LabelEncoder()
    selected_features[col] = le.fit_transform(selected_features[col])
    encoders[col] = le


In [28]:
df['Votes'] = df['Votes'].astype(str).str.replace(',', '', regex=True)
df['Votes'] = pd.to_numeric(df['Votes'], errors='coerce')


In [29]:
df.dropna(subset=['Votes', 'Duration'], inplace=True)


In [30]:
selected_features = df[['Genre', 'Director', 'Actor 1', 'Actor 2', 'Actor 3', 'Year', 'Duration', 'Votes']].copy()
target = df['Rating']


In [31]:
from sklearn.preprocessing import LabelEncoder

encoders = {}
for col in ['Genre', 'Director', 'Actor 1', 'Actor 2', 'Actor 3']:
    le = LabelEncoder()
    selected_features[col] = le.fit_transform(selected_features[col])
    encoders[col] = le


In [32]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(selected_features, target, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)


,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [33]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Model Evaluation Metrics:")
print("MAE:", mae)
print("MSE:", mse)
print("R² Score:", r2)


Model Evaluation Metrics:
MAE: 1.0500166707009388
MSE: 1.6851436641605029
R² Score: 0.08996924726527422


In [34]:
# Example: predicting on the first 5 rows of X_test
predictions = model.predict(X_test[:5])
print("Predicted Ratings:", predictions)
print("Actual Ratings:", y_test[:5].values)


Predicted Ratings: [5.70936358 5.69921556 5.92737373 5.72369721 5.35018683]
Actual Ratings: [6.  2.4 3.8 3.8 7.2]


In [35]:
import joblib
joblib.dump(model, 'movie_rating_model.pkl')


['movie_rating_model.pkl']

In [36]:
model = joblib.load('movie_rating_model.pkl')


In [37]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.5, color='blue')
plt.xlabel("Actual Ratings")
plt.ylabel("Predicted Ratings")
plt.title("Actual vs Predicted Ratings")
plt.grid(True)
plt.show()
